# Evaulation

This notebook is setup to help evaluate the runs

In [1]:
%matplotlib inline
from dotmap import DotMap
from envs import *
import os
import dill as pickle
import run_learning
import numpy as np
import run_eval
import run_render
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
from stable_baselines3 import SAC

# Find Desired Runs

In [2]:
def look_through_folders():
    for folder in os.listdir("./Runs"):
        folder_param_path = os.path.join(os.path.join("./Runs", folder), "params.pkl")
        if os.path.exists(folder_param_path):
            with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
                params = pickle.load(f)
            print("Eps: ", str(params.eps), ", Gamma: ", str(params.gamma), ", LearningRate: ", str(params.learning_rate))
            print("Eval_Freq: ", str(params.init_state))
#look_through_folders()

In [3]:
def check(params):
    match = bool(1
        and params.id == 13
        #and params.finished == True
        #and params.envs.manipulator.cost_func == 3
        )
        

    return match

def filt(check):
    matches = dict()
    for folder in os.listdir("./Runs"):
        folder_param_path = os.path.join(os.path.join("./Runs", folder), "params.pkl")
        if os.path.exists(folder_param_path):
            with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
                params = pickle.load(f)
            if check(params):
                if params.trial_id in matches.keys():
                    matches[params.trial_id].append(folder)
                else:   
                    matches[params.trial_id] = [folder]
    return matches
filt(check)

{8: ['03_19_2022_043427_Mohsin', '03_19_2022_061022_Mohsin'],
 6: ['03_18_2022_234749_Mohsin', '03_18_2022_221305_Mohsin'],
 9: ['03_19_2022_092134_Mohsin', '03_19_2022_074600_Mohsin'],
 3: ['03_18_2022_123559_Mohsin', '03_18_2022_141140_Mohsin'],
 10: ['03_19_2022_123229_Mohsin', '03_19_2022_105730_Mohsin'],
 7: ['03_19_2022_025744_Mohsin', '03_19_2022_012341_Mohsin'],
 2: ['03_18_2022_092553_Mohsin', '03_18_2022_110053_Mohsin'],
 4: ['03_18_2022_172512_Mohsin', '03_18_2022_154757_Mohsin'],
 1: ['03_18_2022_075045_Mohsin', '03_18_2022_061505_Mohsin'],
 11: ['03_19_2022_154352_Mohsin', '03_19_2022_140802_Mohsin'],
 0: ['03_18_2022_030224_Mohsin', '03_18_2022_043952_Mohsin'],
 5: ['03_18_2022_190118_Mohsin', '03_18_2022_203751_Mohsin']}

# Evaluate Desired Runs

### Manipulator

In [ ]:
label = "eps"

def plot_eval():
    plt.figure()
    trial_id = 0
    matches = filt(check)
    for combo in matches.values():
        rew = []
        with open(os.path.join(os.path.join("./Runs", combo[0]), "params.pkl"), 'rb') as f:
            params = pickle.load(f) 
            
        plot = True #plot only one
        for folder in combo:
            results = run_eval.evaluate(folder, env='manipulator', init=[0, 0, 0, 0])
            tb = results["manipulator"]["mean_reward"]
            tb_times = tb[0]
            tb_rew = [a[0] for a in tb[1]]
            
            rew.append(tb_rew)

            if plot:
                theta = results["manipulator"]["thetas"]
                phi = results["manipulator"]["phis"]
                actions = results["manipulator"]["actions"]
            
                plt.subplot(221)
                plt.plot(theta, label=str(params[label]))
                plt.subplot(223)
                plt.plot(phi, label=str(params[label]))
                plt.subplot(224)
                plt.plot(actions, label=str(params[label]))
                plot = False
            
        rew_avg = np.mean(rew, axis=0)        
        n=5
        tb_smoothed = np.cumsum(rew_avg, dtype=float)
        tb_smoothed[n:] = tb_smoothed[n:] - tb_smoothed[:-n]
        tb_smoothed =  tb_smoothed[n - 1:] / n
        
        plt.subplot(222)
        plt.plot(tb_times[(n//2):-(n//2)], tb_smoothed, label=str(params[label]))
        


    plt.subplot(222)
    plt.ylabel("reward")
    plt.legend()
    plt.subplot(221)
    plt.axhline(y=0, color='b', linestyle='--')
    plt.ylabel("theta")
    plt.legend()
    plt.subplot(223)
    plt.axhline(y=0, color='b', linestyle='--')
    plt.ylabel("phi")
    plt.legend()
    plt.subplot(224)
    plt.axhline(y=0, color='b', linestyle='--')
    plt.ylabel("actions")
    plt.legend()
    plt.show()
        
plot_eval()

### Pendulum

In [ ]:
label = "eps"

def plot_eval():
    plt.figure()
    trial_id = 0
    runs = filt(check, trial_id)
    while runs:
        rew = []
        with open(os.path.join(os.path.join("./Runs", runs[0]), "params.pkl"), 'rb') as f:
            params = pickle.load(f) 
        for folder in runs:
            results = run_eval.evaluate(folder, env='newpendulum', init=[1, 0])
            tb = results["newpendulum"]["mean_reward"]
            tb_times = tb[0]
            tb_rew = [a[0] for a in tb[1]]
            
            rew.append(tb_rew)

            theta = results["newpendulum"]["thetas"]
            actions = results["newpendulum"]["actions"]
            
            plt.subplot(221)
            plt.plot(theta, label=str(params[label]))
            plt.subplot(224)
            plt.plot(actions, label=str(params[label]))
            
        rew_avg = np.mean(rew, axis=0)        
        n=5
        tb_smoothed = np.cumsum(rew_avg, dtype=float)
        tb_smoothed[n:] = tb_smoothed[n:] - tb_smoothed[:-n]
        tb_smoothed =  tb_smoothed[n - 1:] / n
        
        plt.subplot(222)
        plt.plot(tb_times[(n//2):-(n//2)], tb_smoothed, label=str(params[label]))
        #plt.plot(tb_times, rew_avg, label=str(params[label]))
        
        trial_id += 1
        runs = filt(check, trial_id)
        


    plt.subplot(222)
    plt.ylabel("reward")
    plt.ylim([-4000, 0])
    plt.legend()
    plt.subplot(221)
    plt.axhline(y=0, color='b', linestyle='--')
    plt.ylabel("theta")
    plt.legend()
    plt.subplot(224)
    plt.axhline(y=0, color='b', linestyle='--')
    plt.ylabel("actions")
    plt.legend()
    plt.show()
        
plot_eval()

# Rendering Station

DON"T USE YET, BEHAVIOR NOT WORKING

In [2]:
folder = '03_19_2022_025744_Mohsin'
env_name = 'manipulator'
model_name = 'best_model'

run_render.render(folder, env_name, model_name)

/opt/anaconda3/envs/rl/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2022-03-20 15:00:41.922 python[99162:6383413] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f9457883420>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-20 15:00:41.922 python[99162:6383413] Warning: Expected min height of view: (<NSButton: 0x7f94578c32d0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-20 15:00:41.924 python[99162:6383413] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f94578b53d0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-20 15:00:41.925 python[99162:6383413] Warning: Expected min he

In [ ]:
folder = '03_19_2022_025744_Mohsin'
env_name = 'manipulator'
model_name = 'best_model'

with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
    params = pickle.load(f)
env = params.envs[env_name].eval_env(params)
env.reset()
models_path = os.path.join(os.path.join("./Runs", folder), env_name + "/models")
model = SAC.load(os.path.join(models_path, model_name))

obs = env.reset()
done = False
i = 0
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
print("hello")
env.close()